# File Search

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    FilePurpose,
    FileSearchTool,
    RunAdditionalFieldList,
    RunStepFileSearchToolCall,
    RunStepToolCallDetails,
    MessageRole
)

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [2]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Setup the file search tool

In [3]:
import os

file_directory = "../data/healthplans"

# List all files in the directory
try:
    filenames = os.listdir(file_directory)
    print(filenames)
except FileNotFoundError:
    print(f"Directory '{file_directory}' not found.")

# Get the full path of a file
def get_filepath_for_filename(filename: str) -> str:
    base_directory = file_directory
    return os.path.join(base_directory, filename)

# Upload files to the project client
file_ids: list[str] = []
for path in [get_filepath_for_filename(filename) for filename in filenames]:
    with open(path, "rb") as file:
        file = project_client.agents.files.upload_and_poll(file_path=path, purpose=FilePurpose.AGENTS)
        print(f"Uploaded file, file ID: {file.id}")
        file_ids.append(file.id)

# Create a vector store with the uploaded files
vector_store = project_client.agents.vector_stores.create_and_poll(file_ids=file_ids, name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")

# Create a file search tool
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

['Northwind_Standard_Benefits_Details.pdf', 'Northwind_Health_Plus_Benefits_Details.pdf']
Uploaded file, file ID: assistant-1Ld2DSXtHLz2itRdH55RdJ
Uploaded file, file ID: assistant-Ev8T7J5XB5S8AoKW7CMNzG
Created vector store, vector store ID: vs_FZxT90salVtx2yOtxn7pp4Vc


## Create the agent

In [4]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-filesearch-agent",  # Name of the agent
    instructions="You are a helpful agent and can search information from uploaded files",  # Instructions for the agent
    tools=file_search.definitions,  # Tools available to the agent
    tool_resources=file_search.resources,  # Resources for the tools
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_eouZucGib9GoM6hnPtY5nrsW


## Create thread

In [5]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_gjNHnFXbKGREN8j68zcXCaGu


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [7]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")


In [8]:
def run_agent_details(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Fetch run steps to see the file search results
    for run_step in project_client.agents.run_steps.list(
        thread_id=thread.id, run_id=run.id, include=[RunAdditionalFieldList.FILE_SEARCH_CONTENTS],
    ):
        # Print the run step details
        if isinstance(run_step.step_details, RunStepToolCallDetails):
            for tool_call in run_step.step_details.tool_calls:
                if (
                    isinstance(tool_call, RunStepFileSearchToolCall)
                    and tool_call.file_search
                    and tool_call.file_search.results
                ): # Check if the tool call is a file search and has results
                    for result in tool_call.file_search.results:
                        if result.content and result.content[0].text:
                            print(
                                "The search tool has found the next relevant content in "
                                f"the file {result.file_name}:"
                            )
                            print(result.content[0].text)
                            print("===============================================================")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")



## Run the agent

In [9]:
user_input = "Provide details on the Northwind Standard Plan"
run_agent_details(user_input)

Created message, ID: msg_M49ISJUZ8zaBdY3TUpX3IkOn
Run finished with status: completed
The search tool has found the next relevant content in the file Northwind_Standard_Benefits_Details.pdf:
Member Services department at 1-800-123-4567. 

It is important to note that Northwind Health is not responsible for any services or benefits 

provided to you by any other health care provider or plan. For example, if you received a 

service or benefit from a non-network provider, Northwind Health will not be responsible 

for any costs associated with that service or benefit. 

In some cases, you may have the right to an external review of your appeal or grievance. If 

you have exhausted the appeals process within Northwind Health and you believe that your 

appeal or grievance was not adequately addressed, you can contact your state's insurance 

department or the federal Department of Health and Human Services for more information 

about the external review process. 

OTHER INFORMATION ABOUT

In [10]:
user_input = "Provide a tabular comparison between the Northwind Standard and Northwind Healthplus plans"
run_agent(user_input)

Created message, ID: msg_LtJ0PwRq74w5RxUQKVmCYXzZ
Run finished with status: completed
Agent Response: Here is a tabular comparison between the Northwind Standard Plan and the Northwind Healthplus Plan:

| Feature                                | Northwind Standard Plan                            | Northwind Healthplus Plan                           |
|----------------------------------------|----------------------------------------------------|----------------------------------------------------|
| **Coverage**                           | Medical, Vision, Dental                            | Medical, Vision, Dental, Prescription Drugs, Mental Health and Substance Abuse, Emergency Services【4:1†source】【4:9†source】【4:10†source】【4:14†source】【4:19†source】               |
| **Deductible**                         | $2,000 per person per year                         | $1,000 per person per year【4:13†source】【4:17†source】                                                             |
| **Out-of-Po

## Cleanup resources

In [11]:
# Cleanup resources

# Delete the vector store after use
project_client.agents.vector_stores.delete(vector_store.id)
print("Deleted vector store")

# Delete the files after use
project_client.agents.files.delete(file_id=file.id)
print("Deleted file")

# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")


Deleted vector store
Deleted file
Deleted agent
